In [ ]:
import pandas as pd
import numpy as np
import pickle

pd.set_option("display.max_columns",None)

In [ ]:
flights = pd.read_csv("../Data/flights.csv")

In [ ]:
len(flights)

In [ ]:
flights.dropna(subset = ["ORIGIN_AIRPORT"], inplace = True)
len(flights)

In [ ]:
flights["ORIGIN_AIRPORT"] = flights["ORIGIN_AIRPORT"].astype(str)
flights["DESTINATION_AIRPORT"] = flights["DESTINATION_AIRPORT"].astype(str)

In [ ]:
flights["code"] = flights["ORIGIN_AIRPORT"] + flights["DESTINATION_AIRPORT"]

In [ ]:
# Buscamos aquellos códigos de aeropuerto que no sean con IATA_CODE tipo str
unknown = flights[flights["code"].str.isdigit()].copy()
unknown.head()

In [ ]:
# Hay que clasificar 306 aeropuertos
unknown['ORIGIN_AIRPORT'].nunique()

In [ ]:
unknown = unknown.sort_values('ORIGIN_AIRPORT')
unknown['ORIGIN_AIRPORT'].unique()

In [ ]:
# Ningun trayecto da lugar a dos distancias diferentes
unknown.groupby("code")["DISTANCE"].nunique().max()

In [ ]:
# Diccionario que relaciona un trayecto y su distancia
dict_flight_distance = dict(zip(unknown["code"],unknown["DISTANCE"]))
dict_flight_distance

Busco IATA_CODE de aeropuerto

In [ ]:
pocho = flights[flights["ORIGIN_AIRPORT"] == '12255']
pocho.head()

# siempre a aeropuerto destino: 11292
#distance = 298 

In [ ]:
pocho_dist = flights[flights["DISTANCE"] == 298]
pocho_dist

In [ ]:
print(pocho_dist['ORIGIN_AIRPORT'].unique())
print(pocho_dist['DESTINATION_AIRPORT'].unique())

In [ ]:
# Voy a tratar de identificar esta ruta '1474711298'

origen = '14747'
destino = '11298'

# Paso 1: Busco todas las rutas con distancia exactamente igual a 1660

# Pongo que el destino tampoco sea aeropuerto de salida para evitar el trayecto de misma distancia y origen distinto,
# Es decir, el trayecto que sale del aeropuerto 11298 y llega al aeropuerto 14747
rutas = flights[(flights["DISTANCE"] == 1660) & (flights["ORIGIN_AIRPORT"] != origen) & (flights["ORIGIN_AIRPORT"] != destino)].copy()
rutas["ORIGIN_AIRPORT"].value_counts()

Busco un codigo de ruta (origen+destino) asociado al orgigen numerico 14747 pero no al destino numerico

In [ ]:
# Con este primer paso tengo ya dos claves que puedo asociar a dos numeros distintos, ahora solo falta saber cual es cual

# Paso 2 Busco otro vuelo que comience con el aeropuerto 14747, voy a elegir solo 1

[path for path in dict_flight_distance.keys() if (path[:5]== origen) & (path[-5:] != destino)][0]

In [ ]:
# Ahora voy a buscar las rutas alternativas que tengo desde ese aeropuerto
alternativa = flights[(flights["DISTANCE"] == dict_flight_distance['1474713487']) & (flights["ORIGIN_AIRPORT"] != origen) & (flights["ORIGIN_AIRPORT"] != '13487')].copy()
alternativa["ORIGIN_AIRPORT"].value_counts()

En este momento, estaría en condiciones de deducir que el código 14747 es el aeropuerto SEA, el código 11298 sería el DFW y
el código 13487 se lo asociaria a MSP 

In [ ]:
# Con este esquema vamos a intentar montar un algoritmo que repita este proceso en todas las rutas disponibles, considerando
# que hay un poco (bastante) más de complejidad en las casuísticas.

# Tenemos por una parte una lista de rutas y tenemos un diccionario que relaciona la ruta con su distancia
dict_flight_distance = dict(zip(unknown["code"],unknown["DISTANCE"]))
paths = [path for path in dict_flight_distance.keys()]

# Creamos una estructura para almacenar los resultados, en este caso un diccionario clave:valor --> numero:aeropuerto
dict_airport = {}

# Elementos base que vamos a utilizar en el algoritmo
path = paths[0]

# Hasta que no sea capaz de clasificar todas las rutas no voy a parar
while paths != []:
    print(path)
    origen = path[:5]
    destino = path[-5:]
    path_inverso = destino + origen
    conjunto_paths = set(paths)
    
    # Primer caso: El origen no se encuentra en el diccionario, es decir, no hay un aeropuerto asociado
    if origen not in dict_airport.keys():
        
        # Busco las opciones de trayecto que me puedo encontrar con esta distancia
        ruta = flights[(flights["DISTANCE"] == dict_flight_distance[path]) & (flights["ORIGIN_AIRPORT"] != origen) & (flights["ORIGIN_AIRPORT"] != destino)].copy()
        opciones = [airport for airport in ruta["ORIGIN_AIRPORT"].unique().tolist() if (not airport.startswith('1')) and (airport not in dict_airport.values())]
        print(f"Las opciones para el origen de la ruta {path} son: {opciones}")
        
        # Si solo existe una opcion para el origen, se la asigno directamente
        # Esto solo puede pasar si el destino ya esta asignado por un path previo
        if len(opciones) == 1:
            dict_airport[origen] = opciones[0]
            
            # Elimino el path resultante ya que origen y destino están ya en el diccionario 
            paths.remove(path)
            
            # Si existe el camino inverso también lo voy a quitar puesto que no merece la pena compararlo
            if path_inverso in paths:
                paths.remove(path_inverso)

        # Si existieran dos o más opciones para el origen buscamos en rutas alternativas posibles puntos comunes
        # como en el ejemplo de arriba
        elif len(opciones) >= 2:
            rutas_alternativas = [path for path in paths if (path[:5]== origen) & (path[-5:] != destino)]

            # Si no existieran rutas alternativas hay que saltar este caso porque tampoco podría encontrar el match de momento
            if rutas_alternativas == []:
                print("No existen más rutas alternativas para este path {}".format(path))
                paths.remove(path)
                paths.append(path)
                path = paths[0]
                continue

            # Busco una ruta alternativa que me permita hacer el match
            
            # Estas dos constantes me sirven para dar cierta seguridad al match, no sirve con tener una ruta alternativa
            # coincidente si no que voy a mirar algunas más
            counter = 0
            coincidencias_requeridas = np.ceil(len(rutas_alternativas)/4)
            for i in range(len(rutas_alternativas)):
                ruta_alternativa = rutas_alternativas[i]
                alternativa = flights[(flights["DISTANCE"] == dict_flight_distance[ruta_alternativa]) & (flights["ORIGIN_AIRPORT"] != origen) & (flights["ORIGIN_AIRPORT"] != ruta_alternativa[-5:])].copy()
                posibilidades = [airport for airport in alternativa["ORIGIN_AIRPORT"].unique().tolist() if (not airport.startswith('1')) and (airport not in dict_airport.values())]
                print(f"Las posibilidades encontradas para el origen de la ruta alternativa {ruta_alternativa} son: {posibilidades}")

                coincidencias = [airport for airport in posibilidades if airport in opciones]
                print(f"Las coincidencias encontradas para el origen en la ruta alternativa {ruta_alternativa} son: {coincidencias}")
                if len(coincidencias) == 1:
                    counter += 1
                    if counter >= coincidencias_requeridas:
                        break
                else:
                    continue
            
        
            print(f"La ruta alternativa final es: {ruta_alternativa}")    

            # Si tras recorrer todas las rutas, no puedo encontrar una que me permita el match, tengo que saltar
            # a otro path, puesto que este no tendría seguridad para inferirlo
            if len(coincidencias) != 1:
                print("No se puede clasificar la ruta {} de momento, por no tener seguridad suficiente con la que hacer match".format(path))
                paths.remove(path)
                paths.append(path)
                path = paths[0]
                continue
         
            # En caso de poder hacer un match para el origen, busco la coincidencia y asigno 
            nuevo_destino = ruta_alternativa[-5:]
            ruta_alternativa_inversa = nuevo_destino + origen

            dict_airport[origen] = coincidencias[0]

            # Si la ruta me lo permite, busco clasificar la otra opción (siempre y cuando haya exactamente dos posibilidades)
            if (destino not in dict_airport.keys()) and (len(opciones) == 2):
                dict_airport[destino] = [airport for airport in opciones if airport != coincidencias[0]][0]
                
                paths.remove(path)
                
                if path_inverso in paths:
                    paths.remove(path_inverso)
            
            # De nuevo, si la ruta alternativa me lo permite, intento clasificar este segundo destino (debe haber exactamente
            # dos posibilidades)
            if (nuevo_destino not in dict_airport.keys()) and (len(posibilidades) == 2):
                dict_airport[nuevo_destino] = [airport for airport in posibilidades if airport != coincidencias[0]][0]

                paths.remove(ruta_alternativa)

                if ruta_alternativa_inversa in paths:
                    paths.remove(ruta_alternativa_inversa)
            
        
    # Segundo caso: Me falta por saber el destino del path    
    elif destino not in dict_airport.keys():
        
        # En este caso entendemos que el origen ya existe dentro del diccionario por lo que realizo la búsqueda para el 
        # destino solamente
        ruta = flights[(flights["DISTANCE"] == dict_flight_distance[path]) & (flights["ORIGIN_AIRPORT"] != origen) & (flights["ORIGIN_AIRPORT"] != destino)].copy()
        opciones = [airport for airport in ruta["ORIGIN_AIRPORT"].unique().tolist() if (not airport.startswith('1')) and (airport not in dict_airport.values())]
        print(f"Las opciones para el destino de la ruta {path} son: {opciones}")
        
        # Razono de la misma manera, si las opciones son más de dos busco una ruta alternativa
        if len(opciones) >= 2:
            # Busco una ruta alternativa que me permita hacer el match con respecto al destino del path
            rutas_alternativas = [path for path in paths if (path[:5]== destino) & (path[-5:] != origen)]
            
            # Si no existieran rutas alternativas hay que saltar este caso porque tampoco podría encontrar el match de momento
            if rutas_alternativas == []:
                print("No existen más rutas alternativas para este path {}".format(path))
                paths.remove(path)
                paths.append(path)
                path = paths[0]
                continue

            counter = 0
            coincidencias_requeridas = np.ceil(len(rutas_alternativas)/4)
            for i in range(len(rutas_alternativas)):
                ruta_alternativa = rutas_alternativas[i]
                alternativa = flights[(flights["DISTANCE"] == dict_flight_distance[ruta_alternativa]) & (flights["ORIGIN_AIRPORT"] != destino) & (flights["ORIGIN_AIRPORT"] != ruta_alternativa[-5:])].copy()
                posibilidades = [airport for airport in alternativa["ORIGIN_AIRPORT"].unique().tolist() if (not airport.startswith('1')) and (airport not in dict_airport.values())]
                print(f"Las posibilidades encontradas para el destino en la ruta alternativa {ruta_alternativa} son: {posibilidades}")
                coincidencias = [airport for airport in posibilidades if airport in opciones]
                print(f"Las coincidencias encontradas para el destino en la ruta alternativa {ruta_alternativa} son: {coincidencias}")
                
                if len(coincidencias) == 1:
                    counter += 1
                    if counter >= coincidencias_requeridas:
                        break
                else:
                    continue
            
            print(f"La ruta alternativa final es: {ruta_alternativa}")   
        
            # Si tras recorrer todas las rutas, no puedo encontrar una que me permita el match, tengo que saltar a otra
            if counter < coincidencias_requeridas:
                print("No se puede clasificar la ruta {} de momento, por no encontrar una ruta alternativa con la que hacer match".format(path))
                paths.remove(path)
                paths.append(path)
                path = paths[0]
                continue
                
            dict_airport[destino] = coincidencias[0]
        
        # Si solo existiera una opcion, se la agrego directamente
        else:
            dict_airport[destino] = opciones[0]

        paths.remove(path)

        if path_inverso in paths:
            paths.remove(path_inverso)
            
    # En el caso de que origen y destino ya hayan sido clasificados por otras rutas diferentes, esta comparación no
    # tiene sentido y solo necesitamos quitarla
    # Ejemplo de este caso: Tengo dos rutas, una que va de A a B y que ha sido clasificada, otra que va de C a D que también
    # ha sido clasificada. Si luego me encuentro la ruta que va de A a C, no necesito clasificarla.
    else:
        paths.remove(path)
        
        if path_inverso in paths:
            paths.remove(path_inverso)
            
    # Mensajes de estado del proceso    
    aeropuertos_clasificados = len(dict_airport.keys())
    rutas_por_clasificar = len(paths)
    path = paths[0]
    print("Aeropuertos clasificados: {}".format(aeropuertos_clasificados))
    print("Rutas por clasificar: {}".format(rutas_por_clasificar))
    
    if len(paths) % 20 == 0:
        print(dict_airport)    

In [ ]:
dict_airport

In [ ]:
dict_airport.values()

In [ ]:
with open('dict_airport.json', 'wb') as fp:
    pickle.dump(dict_airport, fp)